## Preprocess and save training data

In [1]:
import sys
sys.path.append("..")

from tree_parser import file_contents, meta_math_database

# from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps
# from my_utils import get_proof_steps_graph, print_proof_steps_graph

import json


from expanding import construct_proof, expand_all_nodes_with_depth

# from collections import defaultdict

# import glob

# import os

# import numpy as np

# from sklearn.metrics import roc_auc_score

# import random

from tqdm import tqdm

import time

# from expanding import PStep, construct_proof
# from theorem_database import TheoremDatabase

In [2]:
%%time

text = file_contents("../set_mod.mm")
database = meta_math_database(text,n=8000)

included 3651825 tokens from ../set_mod.mm
proposition: 8000CPU times: user 2min 20s, sys: 29.2 s, total: 2min 49s
Wall time: 3min 19s


In [8]:
def get_prop_dataset(prop_label):
    prop = database.propositions[prop_label]
    
    prop_dataset = {
        'id':prop.number,
        'prop': prop_label,
        'steps':[],
        'links':[]
    }
    
    prop_proof = construct_proof(prop)
    prop_proof = expand_all_nodes_with_depth(prop_proof)

    #Populate step numbers
    next_step_n = 0
    
    for s in prop_proof.get_steps_df(): 
        next_step_n += 1
        s._step_num = next_step_n
    
    #Populate data
    
    _no_zero_labels = True 
    #This flag is necessary to remove from the dataset propositions which every label is true
    #All labels true doesnt contribute much to the train/validation
    
    for s in prop_proof.get_steps_df():
        #Since negative class is some depth bigger than 0, in case we have some, set this to false
        if s.statement_depth > 0: _no_zero_labels = False
        
        prop_dataset["steps"].append((
            s._step_num,
            s.label,
            s.raw_statement,
            s.raw_prop_statement,
            int(s.statement_depth == 0)
        ))
        
        for child_s in s.inputs:
            prop_dataset["links"].append((child_s._step_num, s._step_num))
        
    if _no_zero_labels:
        raise Exception(f"Only positive classes in {prop_label}.")
        
    return prop_dataset

In [9]:
props_data = []
failed_props = []

for i, p in enumerate(database.propositions.keys()):
    print(f"\r{i+1}/{len(database.propositions)}", end="")
    
    if len(database.propositions[p].entails_proof_steps) == 0:
        continue
        
    try:
        props_data.append(get_prop_dataset(p))
    except:
        failed_props.append(p)
        
print(len(props_data))
print(len(failed_props))

8000/80007476
154


In [ ]:
#failed_props

In [10]:
ts = time.time()

out_file = open(f"prop_dataset_{ts}.json", "w")
  
json.dump(props_data, out_file)
  
out_file.close()